In [1]:
import pandas as pd
from data_cleaning_util import *

# Dataset itineraries-CostieraAmalfitana

In [2]:
itineraries_costiera_amalfitana_df = pd.read_csv('../Dataset/itineraries-CostieraAmalfitana.csv')

In [3]:
itineraries_costiera_amalfitana_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   NOME               65 non-null     object
 1   LOCALITAˈ          65 non-null     object
 2   TIPOLOGIA          65 non-null     object
 3   GEOLOCALIZZAZIONE  65 non-null     object
 4   DESCRIZIONE        65 non-null     object
 5   NOTE               56 non-null     object
 6   URL IMMAGINE       9 non-null      object
dtypes: object(7)
memory usage: 3.7+ KB


In [4]:
itineraries_costiera_amalfitana_df.head(5)

,NOME,LOCALITAˈ,TIPOLOGIA,GEOLOCALIZZAZIONE,DESCRIZIONE,NOTE,URL IMMAGINE
0,Limone Costa d'Amalfi I.G.P.,"Amalfi, Cetara, Conca dei Marini, Furore, Maio...",Prodotto tipico,"40.63441246747971,14.602812230587006","Il nome della varietà Sfusato Amalfitano, che ...",NaN,http://spod.databenc.it/ethersheet/images/ae8a...
1,Delizia al limone,"Amalfi, Cetara, Conca dei Marini, Furore, Maio...",Prodotto tipico,"40.633594393730164,14.60260570049286",La delizia al limone è un dolce relativamente ...,NaN,http://spod.databenc.it/ethersheet/images/ae8a...
2,Liquore concerto,Tramonti,Prodotto tipico,"40.6997332,14.6415366",E' uno dei rosoli più antichi della Costiera A...,NaN,http://spod.databenc.it/ethersheet/images/ae8a...
3,Fusillo furitano,Minori,Prodotto tipico,"40.650551319122314,14.626602083444595","I ""ricci"" o ""riccioli"", anche detti ""fusilli f...",NaN,http://spod.databenc.it/ethersheet/images/ae8a...
4,Ndunderi,Minori,Prodotto tipico,"40.652708,14.624941",Col nome di ndunderi si indica un tipo di past...,NaN,http://spod.databenc.it/ethersheet/images/ae8a...


In [5]:
print(f"Number of columns {len(itineraries_costiera_amalfitana_df.columns)}\n{itineraries_costiera_amalfitana_df.columns})")

Number of columns 7
Index(['NOME', 'LOCALITAˈ', 'TIPOLOGIA', 'GEOLOCALIZZAZIONE', 'DESCRIZIONE',
       'NOTE', 'URL IMMAGINE'],
      dtype='object'))


## Correction to do:

1. Name - NOME -  keep it as String. Extra spaces might be removed in the data cleansing phase
2. Location - LOCALITA’ - align it with official municipality names (e.g., https://www.istat.it/storage/codici-unita-amministrative/Elenco-comuni-italiani.xls) + deal with multiple values
3. Type - TIPOLOGIA - check value consistency
4. Localization - GEOLOCALIZZAZIONE - consider the possibility to split it in longitude and latitude + replace NON DISPONIBILE/empty values with NULL + check suspicious values
5. Description - DESCRIZIONE - replace NON DISPONIBILE/empty values with NULL + clean values (e.g., numbers)
6. Note - NOTE - replace NON DISPONIBILE/empty values with NULL + clean values (e.g., numbers)
7. Image - URL IMMAGINE -  replace NON DISPONIBILE/empty values with NULL 

### Fill Na with NULL

In [6]:
itineraries_costiera_amalfitana_df = itineraries_costiera_amalfitana_df.fillna(NULL_VALUE)

### 1. Name - NOME -  keep it as String. Extra spaces might be removed in the data cleansing phase

In [7]:
itineraries_costiera_amalfitana_df['NOME'] = itineraries_costiera_amalfitana_df['NOME'].apply(lambda x: remove_extra_spaces(x))
itineraries_costiera_amalfitana_df['NOME'] = itineraries_costiera_amalfitana_df['NOME'].replace(to_replace=["Sant' Angelo dell’Ospedale"], value="Sant'Angelo dell’Ospedale")
itineraries_costiera_amalfitana_df['NOME'].value_counts().sort_index()

Castello di Montalto                               1
Castello di San Nicola de Thoro - Plano            1
Chiesa della Santissima Trinità                    1
Colatura di alici di Cetara                        1
Delizia al limone                                  1
                                                  ..
Torre di Salicerchio                               1
Torre di Santo Stefano                             1
Torre di Vietri, della Marina o di Vito Bianchi    1
Torre la Cerniola                                  1
Torre la Torricella                                1
Name: NOME, Length: 63, dtype: int64

### 2. Location - LOCALITA’ - align it with official municipality names + deal with multiple values

In [8]:
#we substitude the list of municipalities with Amalfi, because also the lon,lat is based in Amalfi
itineraries_costiera_amalfitana_df['LOCALITAˈ'] = itineraries_costiera_amalfitana_df['LOCALITAˈ'].replace(to_replace=["Amalfi, Cetara, Conca dei Marini, Furore, Maiori, Minori, Positano, Praiano, Ravello, Scala, Tramonti, Vietri sul Mare", "Tovere di Amalfi"], value="Amalfi")
#we substitude Vietri, cause there are more municipalities with Vietri in the official name
itineraries_costiera_amalfitana_df['LOCALITAˈ'] = itineraries_costiera_amalfitana_df['LOCALITAˈ'].replace(to_replace=["Vietri"], value="Vietri sul Mare")
itineraries_costiera_amalfitana_df['LOCALITAˈ'] = itineraries_costiera_amalfitana_df['LOCALITAˈ'].replace(to_replace=["Erchie"], value="Maiori")
itineraries_costiera_amalfitana_df['LOCALITAˈ'] = itineraries_costiera_amalfitana_df['LOCALITAˈ'].apply(lambda x: municipality_alignment(comune=x,\
                                                                                comuni_istat_ita_list=comuni_istat_ita_list,\
                                                                                comuni_istat_altra_lingua_list=comuni_istat_altra_lingua_list,\
                                                                                comuni_istat_ita_e_straniero_list=comuni_istat_ita_e_straniero_list,\
                                                                                null_value=NULL_VALUE, \
                                                                                max_levenshtein_distance=2))

itineraries_costiera_amalfitana_df['LOCALITAˈ'].value_counts().sort_index()

Amalfi               8
Atrani               4
Cetara               2
Conca dei Marini     4
Furore               2
Maiori              15
Minori               6
Positano             7
Praiano              4
Ravello              4
Scala                2
Tramonti             2
Vietri sul Mare      5
Name: LOCALITAˈ, dtype: int64

### 3. Type - TIPOLOGIA - check value consistency

In [9]:
itineraries_costiera_amalfitana_df['TIPOLOGIA'].value_counts().sort_index()

Castello            2
Chiesa rupestre    10
Prodotto tipico     9
Torre               2
Torre costiera     42
Name: TIPOLOGIA, dtype: int64

### 4. Localization - GEOLOCALIZZAZIONE - consider the possibility to split it in longitude and latitude + replace NON DISPONIBILE/empty values with NULL + check suspicious values

In [10]:
temp_separator = ";;"
itineraries_costiera_amalfitana_df['Temp'] = itineraries_costiera_amalfitana_df['GEOLOCALIZZAZIONE'].apply(lambda x: temp_separator.join(float_tuple_to_string_list(get_latitude_longitude(string=x, null_value=NULL_VALUE))))
itineraries_costiera_amalfitana_df[['LATITUDE', 'LONGITUDE']] = itineraries_costiera_amalfitana_df.Temp.str.split(temp_separator, expand=True)
itineraries_costiera_amalfitana_df = itineraries_costiera_amalfitana_df.drop(['Temp'], axis=1)

itineraries_costiera_amalfitana_df.drop_duplicates(subset = ['LATITUDE', 'LONGITUDE'])[['LATITUDE', 'LONGITUDE']]

,LATITUDE,LONGITUDE
0,40.63441246747971,14.602812230587006
1,40.633594393730164,14.60260570049286
2,40.6997332,14.6415366
3,40.650551319122314,14.626602083444595
4,40.652708,14.624941
...,...,...
60,40.66944479942322,14.732317328453064
61,40.65782278776169,14.712235629558563
62,40.65788850188255,14.712146446108818
63,40.66430568695069,14.71624553203583


### 5. Description - DESCRIZIONE - replace NON DISPONIBILE/empty values with NULL + clean values (e.g., numbers)

In [11]:
itineraries_costiera_amalfitana_df['DESCRIZIONE'].value_counts().sort_index()

Cetara è un antico borgo di pescatori in provincia di Salerno, ancora oggi caratterizzato da un'economia concentrata sulla pesca e sull'artigianato. A Cetara continua a essere presente nella tradizione culinaria, la colatura di alici, erede del Garum, un piatto descrittoci da Plinio e Orazio come una salsa di pesce cremosa che veniva ottenuta facendo macerare strati alternati di pesci piccoli e interi, probabilmente alici e pesci più grandi tagliati a pezzetti, forse sgombri o tonni, con strati di erbe aromatiche tritate, tutto ricoperto da sale grosso. La colatura di alici che viene prodotta a Cetara è un liquido ambrato ottenuto seguendo un antico procedimento che i pescatori del luogo si sono tramandati di padre in figlio. Si parte dalla tecnica di lavorazione delle alici sotto sale, di cui la colatura è un derivato: le alici appena pescate, in tutto il periodo primaverile, vengono private della testa e delle interiora e poi adagiate in un contenitore, cosparse di sale marino abbond

### 6. Note - NOTE - replace NON DISPONIBILE/empty values with NULL + clean values (e.g., numbers)

In [12]:
itineraries_costiera_amalfitana_df['NOTE'] = itineraries_costiera_amalfitana_df['NOTE'].str.capitalize()
itineraries_costiera_amalfitana_df['NOTE'] = itineraries_costiera_amalfitana_df['NOTE'].str.replace("vicerale", "vicereale", regex=False)
itineraries_costiera_amalfitana_df['NOTE'] = itineraries_costiera_amalfitana_df['NOTE'].str.replace(NULL_VALUE.capitalize(), NULL_VALUE, regex=False)
itineraries_costiera_amalfitana_df['NOTE'].value_counts().sort_index()

Chiamata anche americana o del mortaro. epoca vicereale                                                                                                                                                                                                  1
Chiamata anche torre revigliano o revellino. epoca vicereale                                                                                                                                                                                             1
Conosciuta anche col nome di torretta di capo d'orso. epoca angioina                                                                                                                                                                                     1
Detta anche della formicola. epoca vicereale                                                                                                                                                                                                           

### 7. Image - URL IMMAGINE -  replace NON DISPONIBILE/empty values with NULL 

In [13]:
itineraries_costiera_amalfitana_df['URL IMMAGINE'].value_counts().sort_index()

NULL                                                                                                                            56
http://spod.databenc.it/ethersheet/images/ae8a366d-e0fa-405c-8d1e-882d84b56df8/18721305_10211478306570096_1522314104_o.jpg       1
http://spod.databenc.it/ethersheet/images/ae8a366d-e0fa-405c-8d1e-882d84b56df8/Delizia_al_limone.jpg                             1
http://spod.databenc.it/ethersheet/images/ae8a366d-e0fa-405c-8d1e-882d84b56df8/Limoncello.jpg                                    1
http://spod.databenc.it/ethersheet/images/ae8a366d-e0fa-405c-8d1e-882d84b56df8/Limone_di_Sorrento.jpg                            1
http://spod.databenc.it/ethersheet/images/ae8a366d-e0fa-405c-8d1e-882d84b56df8/Marroni_.jpeg                                     1
http://spod.databenc.it/ethersheet/images/ae8a366d-e0fa-405c-8d1e-882d84b56df8/Progetto senza titolo (2).png                     1
http://spod.databenc.it/ethersheet/images/ae8a366d-e0fa-405c-8d1e-882d84b56df8/Sfog

### 8. Address - adding columns reporting zip code, municipality, country, etc.

In [14]:
temp_separator = ";;"
itineraries_costiera_amalfitana_df['Temp'] = itineraries_costiera_amalfitana_df['LOCALITAˈ'].apply(lambda x: temp_separator.join(getAddressInfo(comune=map_comune_to_elenco_comuni_cap(x), elenco_comuni_cap=elenco_comuni_cap)))
itineraries_costiera_amalfitana_df[['CAP','PROVINCIA','REGIONE']] = itineraries_costiera_amalfitana_df.Temp.str.split(temp_separator, expand=True)
itineraries_costiera_amalfitana_df = itineraries_costiera_amalfitana_df.drop(['Temp'], axis=1)

itineraries_costiera_amalfitana_df.drop_duplicates(subset = ['LOCALITAˈ', 'CAP','PROVINCIA','REGIONE'])[['LOCALITAˈ', 'CAP','PROVINCIA','REGIONE']]

,LOCALITAˈ,CAP,PROVINCIA,REGIONE
0,Amalfi,84011,SA,Campania
2,Tramonti,84010,SA,Campania
3,Minori,84010,SA,Campania
5,Scala,84010,SA,Campania
7,Cetara,84010,SA,Campania
8,Conca dei Marini,84010,SA,Campania
10,Atrani,84010,SA,Campania
12,Furore,84010,SA,Campania
13,Maiori,84010,SA,Campania
16,Ravello,84010,SA,Campania


## Define type of itinerary

In [15]:
itinerary_culinare_name_list = ["Limone Costa d'Amalfi I.G.P.","Delizia al limone", \
                                "Liquore concerto", "Fusillo furitano", \
                                "Ndunderi", "Marrone di Scala", \
                                "Liquore di limone Costa d'Amalfi", "Colatura di alici di Cetara", \
                                "Sfogliatella Santa Rosa"]

itineraries_costiera_amalfitana_df["CATEGORIA ITINERARIO"] = 'culturale'
itineraries_costiera_amalfitana_df.loc[itineraries_costiera_amalfitana_df.NOME.isin(itinerary_culinare_name_list), "CATEGORIA ITINERARIO"] = 'culinario'

## Save to csv and pickle

In [16]:
itineraries_costiera_amalfitana_df = itineraries_costiera_amalfitana_df[['CATEGORIA ITINERARIO', 'NOME', \
        'LOCALITAˈ', 'CAP', 'PROVINCIA', 'REGIONE', 'TIPOLOGIA', 'DESCRIZIONE', 'NOTE', 'URL IMMAGINE', \
        'LATITUDE', 'LONGITUDE']]
itineraries_costiera_amalfitana_df.to_csv('../Dataset_clean/itineraries-CostieraAmalfitana.csv', index=False)
itineraries_costiera_amalfitana_df.to_pickle("../Dataset_clean/itineraries-CostieraAmalfitana.pkl")  